In [1]:
#Start of code for project 6

In [8]:
import pandas as pd
import seaborn as sns

In [9]:
df = pd.read_csv('../data/exchange-2_cpc_results.csv')

In [10]:
df.head(10)

,timestamp,value,unique_id
0,7/1/2011 0:00,0.081965,1
1,7/1/2011 1:00,0.098972,2
2,7/1/2011 2:00,0.065314,3
3,7/1/2011 3:00,0.070663,4
4,7/1/2011 4:00,0.102490,5
5,7/1/2011 5:00,0.123395,6
6,7/1/2011 6:00,0.153045,7
7,7/1/2011 7:00,0.148322,8
8,7/1/2011 8:00,0.218258,9
9,7/1/2011 9:00,0.226598,10


In [11]:
#From slides, we can first do the Inter-quartile range

In [12]:
from scipy.stats import iqr

In [13]:
iqr(df.value)


0.046985546

In [14]:
iqr(df.value, axis=0)

0.046985546

In [15]:
import numpy as np

In [16]:
np.std(df)

value          0.033718
unique_id    468.808330
dtype: float64

In [17]:
u = iqr(df.value) + 2*np.std(df)

In [18]:
l = iqr(df.value) - 2*np.std(df)

In [19]:
u


value          0.114422
unique_id    937.663645
dtype: float64

In [20]:
l

value         -0.020451
unique_id   -937.569674
dtype: float64

In [21]:
#This gives the upper and lower bounds for the range. Numbers outside these bounds will be considered anomolies.

In [60]:
#Can also try Naive Bayes which was presented in class which is presented below

In [23]:
from sklearn.naive_bayes import GaussianNB

In [24]:
gnb = GaussianNB()


In [25]:
y_pred = gnb.fit(df.value, df.unique_id).predict(df.data)


ValueError: Expected 2D array, got 1D array instead:
array=[0.08196474 0.09897224 0.06531395 ... 0.09620098 0.08538636 0.10932692].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [26]:
df.pivot(index='timestamp', columns='variable', values='value')


AttributeError: 'DataFrame' object has no attribute 'reshape'

In [57]:
#I could not find a way to solve this problem. For some reason the fit and predict functions do not seem to work with 
#this data. Reshape does not work with DataFrame. When I looked online, I wasn't able to work out exactly how to adjust here. 

In [59]:
#Below is the example (using that iris dataset!!) that is online for how to implement Naive Bayes
#I could get this to work, so there seems to be a difference in the data (the arrays look different)

In [27]:
from sklearn import datasets
iris = datasets.load_iris()
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(iris.data, iris.target).predict(iris.data)


In [30]:
print("Number of mislabeled points out of a total %d points : %d"
...     % (iris.data.shape[0],(iris.target != y_pred).sum()))


Number of mislabeled points out of a total 150 points : 6


In [56]:
#I tried to build a Naive Bayes model from scratch using an oneline source as opposed to 
#using one of the libraries and commands. 

In [52]:
import csv
import random
import math
 
#def loadCsv(filename):
#	lines = csv.reader(open(filename, "rb"))
#	dataset = list(lines)
#	for i in range(len(dataset)):
#		dataset[i] = [float(x) for x in dataset[i]]
#	return dataset
outliers = str(df)

def splitDataset(dataset, splitRatio):
	trainSize = int(len(dataset) * splitRatio)
	trainSet = []
	copy = list(dataset)
	while len(trainSet) < trainSize:
		index = random.randrange(len(copy))
		trainSet.append(copy.pop(index))
	return [trainSet, copy]
 
def separateByClass(dataset):
	separated = {}
	for i in range(len(dataset)):
		vector = dataset[i]
		if (vector[-1] not in separated):
			separated[vector[-1]] = []
		separated[vector[-1]].append(vector)
	return separated
 
def mean(numbers):
	return sum(numbers)/float(len(numbers))
 
def stdev(numbers):
	avg = mean(numbers)
	variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
	return math.sqrt(variance)
 
def summarize(dataset):
	summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
	del summaries[-1]
	return summaries
 
def summarizeByClass(dataset):
	separated = separateByClass(dataset)
	summaries = {}
	for classValue, instances in separated.iteritems():
		summaries[classValue] = summarize(instances)
	return summaries
 
def calculateProbability(x, mean, stdev):
	exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
	return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent
 
def calculateClassProbabilities(summaries, inputVector):
	probabilities = {}
	for classValue, classSummaries in summaries.iteritems():
		probabilities[classValue] = 1
		for i in range(len(classSummaries)):
			mean, stdev = classSummaries[i]
			x = inputVector[i]
			probabilities[classValue] *= calculateProbability(x, mean, stdev)
	return probabilities
			
def predict(summaries, inputVector):
	probabilities = calculateClassProbabilities(summaries, inputVector)
	bestLabel, bestProb = None, -1
	for classValue, probability in probabilities.iteritems():
		if bestLabel is None or probability > bestProb:
			bestProb = probability
			bestLabel = classValue
	return bestLabel
 
def getPredictions(summaries, testSet):
	predictions = []
	for i in range(len(testSet)):
		result = predict(summaries, testSet[i])
		predictions.append(result)
	return predictions
 
def getAccuracy(testSet, predictions):
	correct = 0
	for i in range(len(testSet)):
		if testSet[i][-1] == predictions[i]:
			correct += 1
	return (correct/float(len(testSet))) * 100.0
 
def main():
	splitRatio = 0.67
	dataset = outliers
	trainingSet, testSet = splitDataset(dataset, splitRatio)
	print('Split {0} rows into train={1} and test={2} rows'.format(len(dataset), len(trainingSet), len(testSet)))
	# prepare model
	summaries = summarizeByClass(trainingSet)
	# test model
	predictions = getPredictions(summaries, testSet)
	accuracy = getAccuracy(testSet, predictions)
	print('Accuracy: {0}%'.format(accuracy))
 
main()

Split 2628 rows into train=1760 and test=868 rows


AttributeError: 'dict' object has no attribute 'iteritems'

In [53]:
import csv
def loadCsv(filename):
	lines = csv.reader(open(filename, "rb"))
	dataset = list(lines)
	for i in range(len(dataset)):
		dataset[i] = [float(x) for x in dataset[i]]
	return dataset

In [55]:

filename = '../data/exchange-2_cpc_results.csv'
dataset = loadCsv(filename)
print('Loaded data file {0} with {1} rows').format(filename, len(dataset))

Error: iterator should return strings, not bytes (did you open the file in text mode?)